In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score
import random
from datetime import datetime
import multiprocessing as mp
from pytorch_tcn import TCN
# Import the module
from audio_processing import get_file_paths_and_labels, load_and_resample_audio, extract_mfcc, MFCCDataset

# Set multiprocessing start method to 'spawn'
mp.set_start_method('spawn', force=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Function to save and load the model
def save_model(model, path):
    torch.save(model.state_dict(), path)

def load_model(model, path):
    model.load_state_dict(torch.load(path))
    model.to(device)

base_dir = r'/home/gridsan/abradshaw/MIT Buzz'
model_save_path = os.path.join(base_dir, 'saved_tcn_model.pth')

# Get file paths and labels
file_paths, labels = get_file_paths_and_labels(base_dir)

# Collect first five files for each label category
negative_files = [(path, label) for path, label in zip(file_paths, labels) if label == 0][:5]
positive_files = [(path, label) for path, label in zip(file_paths, labels) if label == 1][:5]

print("First five files in negative dataset (label 0):")
for file_path, label in negative_files:
    print(f"File: {file_path}, Label: {label}")

print("\nFirst five files in positive dataset (label 1):")
for file_path, label in positive_files:
    print(f"File: {file_path}, Label: {label}")

print(f"Found {len(file_paths)} audio files.")

# Iterate through all file paths and process each file
for file_path in file_paths:
    y, sr = load_and_resample_audio(file_path)
    if y is not None:
        mfcc_features = extract_mfcc(y, sr)
        if mfcc_features is not None:
            print(f"Extracted MFCCs for file {file_path}. Shape: {mfcc_features.shape}")
        else:
            print(f"Failed to extract MFCCs for file {file_path}.")
    else:
        print(f"Failed to load audio for file {file_path}.")

/home/gridsan/abradshaw/.conda/envs/python39++/lib/python3.9/site-packages/pytorch_tcn/tcn.py:12: UserWarning: 
        The deprecated weight_norm from torch.nn.utils.weight_norm was imported.
        Update your PyTorch version to get rid of this warning.
        
  warnings.warn(


Using device: cuda
First five files in negative dataset (label 0):
File: /home/gridsan/abradshaw/MIT Buzz/negative_dataset_1sec/Native_sample_2914_segment_9.mp3, Label: 0
File: /home/gridsan/abradshaw/MIT Buzz/negative_dataset_1sec/Native_sample_389_segment_11.mp3, Label: 0
File: /home/gridsan/abradshaw/MIT Buzz/negative_dataset_1sec/Invasive_sample_676_segment_1.mp3, Label: 0
File: /home/gridsan/abradshaw/MIT Buzz/negative_dataset_1sec/Invasive_sample_473_segment_18.mp3, Label: 0
File: /home/gridsan/abradshaw/MIT Buzz/negative_dataset_1sec/Native_sample_2717_segment_11.mp3, Label: 0

First five files in positive dataset (label 1):
File: /home/gridsan/abradshaw/MIT Buzz/positive_dataset_1sec/Invasive_sample_892_segment_18.mp3, Label: 1
File: /home/gridsan/abradshaw/MIT Buzz/positive_dataset_1sec/Invasive_sample_872_segment_18.mp3, Label: 1
File: /home/gridsan/abradshaw/MIT Buzz/positive_dataset_1sec/Invasive_sample_819_segment_1.mp3, Label: 1
File: /home/gridsan/abradshaw/MIT Buzz/posi

In [3]:
print("Creating dataset...")
dataset = MFCCDataset(file_paths, labels)
print("Dataset created")

# Split the data into training, validation, and test sets
train_files, temp_files, train_labels, temp_labels = train_test_split(file_paths, labels, test_size=0.4, random_state=42)
val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

train_dataset = MFCCDataset(train_files, train_labels)
val_dataset = MFCCDataset(val_files, val_labels)
test_dataset = MFCCDataset(test_files, test_labels)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


Creating dataset...
Dataset created


In [4]:
class TCNModel(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size, dropout, causal, use_norm, activation):
        super(TCNModel, self).__init__()
        self.tcn = TCN(
            num_inputs=num_inputs,
            num_channels=num_channels,
            kernel_size=kernel_size,
            dropout=dropout,
            causal=causal,
            use_norm=use_norm,
            activation=activation
        )
        self.bn = nn.BatchNorm1d(num_channels[-1])
        self.fc = nn.Linear(num_channels[-1], 1)
        nn.init.kaiming_normal_(self.fc.weight, nonlinearity='relu')

    def forward(self, x):
        x = self.tcn(x)
        x = self.bn(x)
        x = torch.mean(x, dim=2)
        x = self.fc(x)
        return x

# Instantiate the model
model = TCNModel(
    num_inputs=30,
    num_channels=[16, 32, 64, 128],
    kernel_size=5,
    dropout=0.4,
    causal=False,
    use_norm='weight_norm',
    activation='relu'
).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
def train(model, dataloader, criterion, optimizer, device, num_epochs=20, clip_value=1.0):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        all_preds = []
        all_labels = []
        for mfcc_batch, labels_batch in dataloader:
            if mfcc_batch is None:
                continue
            mfcc_batch, labels_batch = mfcc_batch.to(device), labels_batch.unsqueeze(-1).to(device)
            optimizer.zero_grad()
            outputs = model(mfcc_batch)
            loss = criterion(outputs, labels_batch)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            optimizer.step()
            total_loss += loss.item()
            preds = torch.round(torch.sigmoid(outputs)).detach().cpu().numpy()  # Detach before converting to numpy
            all_preds.extend(preds)
            all_labels.extend(labels_batch.cpu().numpy())
        accuracy = accuracy_score(all_labels, all_preds)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}, Accuracy: {accuracy:.4f}")
save_model(model, 'saved_tcn_model.pth')

In [6]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for mfcc_batch, labels_batch in dataloader:
            if mfcc_batch is None:
                continue
            mfcc_batch, labels_batch = mfcc_batch.to(device), labels_batch.unsqueeze(-1).to(device)
            outputs = model(mfcc_batch)
            loss = criterion(outputs, labels_batch)
            total_loss += loss.item()
            preds = torch.round(torch.sigmoid(outputs)).detach().cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels_batch.cpu().numpy())
    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    print(f"Validation - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, Precision: {precision:.4f}")
    return avg_loss, accuracy, all_preds, all_labels


In [8]:
def manual_validation(file_paths, labels, preds, base_dir, num_samples=100):
    correct_files = []
    false_positives = []
    false_negatives = []

    output_dir = os.path.join(base_dir, 'manual_validation_results')
    os.makedirs(output_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file_path = os.path.join(output_dir, f'validation_results_{timestamp}.txt')
    
    for i, (label, pred) in enumerate(zip(labels, preds)):
        file_path = file_paths[i]
        if label == pred:
            correct_files.append((file_path, pred, label))
        elif label == 0 and pred == 1:
            false_positives.append((file_path, pred, label))
        elif label == 1 and pred == 0:
            false_negatives.append((file_path, pred, label))
    
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, zero_division=1)
    precision = precision_score(labels, preds, zero_division=1)
    
    with open(output_file_path, 'w') as f:
        f.write(f"Cross-validation Results:\n")
        f.write(f"Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, Precision: {precision:.4f}\n")
        f.write(f"Total Correct: {len(correct_files)}\n")
        f.write(f"Total False Positives: {len(false_positives)}\n")
        f.write(f"Total False Negatives: {len(false_negatives)}\n\n")
        
        sample_correct = random.sample(correct_files, min(len(correct_files), num_samples))
        sample_false_positives = random.sample(false_positives, min(len(false_positives), num_samples))
        sample_false_negatives = random.sample(false_negatives, min(len(false_negatives), num_samples))
        
        f.write("Sampled Correct Predictions:\n")
        for fp, pred, label in sample_correct:
            f.write(f"File: {fp}, Prediction: {pred}, True Label: {label}\n")
        
        f.write("\nSampled False Positives:\n")
        for fp, pred, label in sample_false_positives:
            f.write(f"File: {fp}, Prediction: {pred}, True Label: {label}\n")
        
        f.write("\nSampled False Negatives:\n")
        for fp, pred, label in sample_false_negatives:
            f.write(f"File: {fp}, Prediction: {pred}, True Label: {label}\n")

    print(f"Validation results saved to {output_file_path}")

In [9]:
# #Run
# print("Training the model...")
# train(model, train_loader, criterion, optimizer, device, num_epochs=20)

# #load the saved model
# load_model(model, model_save_path)

# print("Evaluating the model on validation set...")
# val_loss, val_accuracy, val_preds, val_labels = evaluate(model, val_loader, criterion, device)

# print("Evaluating the model on test set...")
# test_loss, test_accuracy, test_preds, test_labels = evaluate(model, test_loader, criterion, device)

print("Running manual validation...")
manual_validation(test_loader.dataset.file_paths, test_labels, test_preds, base_dir)

Running manual validation...
Validation results saved to /home/gridsan/abradshaw/MIT Buzz/manual_validation_results/validation_results_20240604_164150.txt
